Run all this in colab 

In [ ]:
!pip install flask ngrok transformers torch pillow

In [ ]:
%%writefile app.py
from flask import Flask, request, jsonify
from PIL import Image
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import io

app = Flask(__name__)

# Load BLIP-2 model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
).to(device)

# Generate caption from image
def generate_caption(image):
    prompt = "What is the road damage in this image?"
    image = image.resize((512, 512))
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_new_tokens=100)
    caption = processor.tokenizer.decode(output[0], skip_special_tokens=True).strip()
    if caption.lower().strip() in ["", prompt.lower().strip()]:
        caption = "[Model failed to generate meaningful caption]"
    return caption

# Rule-based summary
def generate_summary(caption):
    caption_lower = caption.lower()
    if "pothole" in caption_lower:
        return "Pothole detected", "High" if "deep" in caption_lower else "Low", "Urgent" if "center" in caption_lower else "Low"
    elif "crack" in caption_lower:
        return "Crack detected", "Medium" if "long" in caption_lower else "Low", "Moderate"
    elif "debris" in caption_lower:
        return "Debris detected", "Low", "Moderate"
    elif "blocked" in caption_lower or "collapsed" in caption_lower:
        return "Roadblock detected", "High", "Critical"
    elif caption == "[Model failed to generate meaningful caption]":
        return "Unknown detected", "Unknown", "Unknown"
    else:
        return "No damage detected", "None", "None"

@app.route('/analyze', methods=['POST'])
def analyze():
    if 'image' not in request.files:
        return jsonify({'error': 'No image uploaded'}), 400

    file = request.files['image']
    try:
        image = Image.open(io.BytesIO(file.read())).convert("RGB")
        caption = generate_caption(image)
        damage_type, severity, priority = generate_summary(caption)

        # Combine summary into a single string
        summary_text = f"{damage_type}. Severity: {severity}. Priority: {priority}."

        return jsonify({
            'caption': caption,
            'summary': summary_text  # Send summary as a string
        })

    except Exception as e:
        return jsonify({'error': 'Failed to analyze image', 'details': str(e)}), 500


In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import conf, ngrok

# Replace this with your actual token
conf.get_default().auth_token = "2vdUtkrbYytKnnqmvVY3LFM6MvG_7kfvaoDVoXm9adX21wDtc"

In [ ]:
from flask import Flask
from pyngrok import ngrok
import os

# Set environment variable for Flask app
os.environ["FLASK_APP"] = "app.py"

# Kill any running tunnels
ngrok.kill()

# Open a tunnel on the default Flask port 5000
public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url)

# Start Flask app
!flask run --host=0.0.0.0 --port=5000
